In [33]:
# Imports
import pandas as pd
import numpy as np
import io
import pytz
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset
import yfinance as yf
import pandas_ta as ta

## Step 1: Create the MACD Trading Algorithm


In [34]:
#import the S&P 500 data

df = pd.read_csv(
    Path("../Resources/spy.csv"),
    index_col = 'Date',
    infer_datetime_format=True,
    parse_dates=True
)

df.head()

,Close,Open,High,Volume,Low
Date,,,,,
2021-04-13 09:30:00-04:00,411.950012,411.529999,412.149994,11677340,411.119995
2021-04-13 10:30:00-04:00,412.059998,411.950012,412.200012,5872073,411.720001
2021-04-13 11:30:00-04:00,411.959991,412.059906,412.209991,3257155,411.739990
2021-04-13 12:30:00-04:00,411.970001,411.964996,412.119995,6086220,411.540009
2021-04-13 13:30:00-04:00,412.698090,411.975006,412.720001,7161962,411.940002


In [35]:
# Filter the date index, open, high, low and close columns
macd_df = df.copy()

macd_df.head()

,Close,Open,High,Volume,Low
Date,,,,,
2021-04-13 09:30:00-04:00,411.950012,411.529999,412.149994,11677340,411.119995
2021-04-13 10:30:00-04:00,412.059998,411.950012,412.200012,5872073,411.720001
2021-04-13 11:30:00-04:00,411.959991,412.059906,412.209991,3257155,411.739990
2021-04-13 12:30:00-04:00,411.970001,411.964996,412.119995,6086220,411.540009
2021-04-13 13:30:00-04:00,412.698090,411.975006,412.720001,7161962,411.940002


In [36]:
# Calculate MACD values using the pandas_ta library
macd_df.ta.macd(close='Close',  fast=12, slow=26, signal=9, append=True)

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,
2021-04-13 09:30:00-04:00,NaN,NaN,NaN
2021-04-13 10:30:00-04:00,NaN,NaN,NaN
2021-04-13 11:30:00-04:00,NaN,NaN,NaN
2021-04-13 12:30:00-04:00,NaN,NaN,NaN
2021-04-13 13:30:00-04:00,NaN,NaN,NaN
...,...,...,...
2022-04-12 12:30:00-04:00,-1.904712,-0.045560,-1.859152
2022-04-12 13:30:00-04:00,-2.125682,-0.213224,-1.912458
2022-04-12 14:30:00-04:00,-2.388655,-0.380958,-2.007697


In [37]:
#drop NaN values

macd_df = macd_df.dropna()

macd_df.head()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,,,,,,
2021-04-19 14:30:00-04:00,414.325012,415.010010,415.195007,9571420,414.220001,0.679881,-0.592633,1.272514
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,14176944,414.140015,0.621316,-0.520959,1.142274
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,11437142,413.660004,0.463053,-0.543377,1.006430
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,17431474,411.119995,0.146716,-0.687771,0.834487
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,10103321,410.619995,-0.057330,-0.713454,0.656124


In [38]:
# set the share size to 100
share_size = 100

In [39]:
#set the initial_capital to 100000

initial_capital = 100000

In [40]:
#Create the MACD signal

macd_df["Signal"]=0.0

macd_df["Signal"] = np.where(
    macd_df["MACDh_12_26_9"] > 0, 1.0, 0.0
)

#calculate entry/exit points, 1 or -1
macd_df["Entry/Exit"] = macd_df["Signal"].diff()

#review the dataframe
macd_df.tail(10)

C:\Users\nmend\AppData\Local\Temp/ipykernel_3172/2548536393.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  macd_df["Signal"]=0.0
C:\Users\nmend\AppData\Local\Temp/ipykernel_3172/2548536393.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  macd_df["Signal"] = np.where(
C:\Users\nmend\AppData\Local\Temp/ipykernel_3172/2548536393.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit
Date,,,,,,,,,,
2022-04-11 14:30:00-04:00,440.959991,441.529999,443.000000,10120550,440.859985,-2.135677,-0.548307,-1.587370,0.0,0.0
2022-04-11 15:30:00-04:00,439.959991,440.950012,441.019989,18938227,439.390015,-2.305212,-0.574274,-1.730938,0.0,0.0
2022-04-12 09:30:00-04:00,443.589996,443.079987,445.750000,18679011,442.369995,-2.122195,-0.313006,-1.809190,0.0,0.0
2022-04-12 10:30:00-04:00,444.059998,443.600006,444.230011,9075047,442.219910,-1.917129,-0.086351,-1.830778,0.0,0.0
2022-04-12 11:30:00-04:00,441.790009,444.070007,444.500000,6698770,441.630005,-1.915698,-0.067937,-1.847762,0.0,0.0
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,7124676,441.170013,-1.904712,-0.045560,-1.859152,0.0,0.0
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,10061518,438.420013,-2.125682,-0.213224,-1.912458,0.0,0.0
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,14347270,436.650085,-2.388655,-0.380958,-2.007697,0.0,0.0
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,12496573,436.679993,-2.470944,-0.370598,-2.100347,0.0,0.0


In [41]:
#create a column named "Position" by multiplying the share_size by the signal
# buy a position when the MACD signal = 2 (MACD histogram shows positive value indicating a bullish cross)
# sell a position when the MACD signal = -2 (MACD histogram shows negative value indicating a bearish cross)

macd_df["Position"] = share_size * macd_df["Signal"]

#review the dataframe
macd_df.head(20)

C:\Users\nmend\AppData\Local\Temp/ipykernel_3172/3563055672.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  macd_df["Position"] = share_size * macd_df["Signal"]


,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position
Date,,,,,,,,,,,
2021-04-19 14:30:00-04:00,414.325012,415.010010,415.195007,9571420,414.220001,0.679881,-0.592633,1.272514,0.0,NaN,0.0
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,14176944,414.140015,0.621316,-0.520959,1.142274,0.0,0.0,0.0
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,11437142,413.660004,0.463053,-0.543377,1.006430,0.0,0.0,0.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,17431474,411.119995,0.146716,-0.687771,0.834487,0.0,0.0,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,10103321,410.619995,-0.057330,-0.713454,0.656124,0.0,0.0,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,5748608,411.250000,-0.284746,-0.752696,0.467950,0.0,0.0,0.0
2021-04-20 13:30:00-04:00,411.535004,411.309692,412.100006,5744918,411.200012,-0.440132,-0.726465,0.286333,0.0,0.0,0.0
2021-04-20 14:30:00-04:00,411.904999,411.429993,412.019989,8414075,410.589996,-0.527342,-0.650940,0.123598,0.0,0.0,0.0
2021-04-20 15:30:00-04:00,412.239990,411.904999,412.670013,13401855,411.799988,-0.562936,-0.549228,-0.013709,0.0,0.0,0.0


In [42]:
#find the points in time where a position is purchased or sold
macd_df["Entry/Exit Position"] = macd_df["Position"].diff()

#review the dataframe
macd_df.head()


C:\Users\nmend\AppData\Local\Temp/ipykernel_3172/700383951.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  macd_df["Entry/Exit Position"] = macd_df["Position"].diff()


,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position
Date,,,,,,,,,,,,
2021-04-19 14:30:00-04:00,414.325012,415.010010,415.195007,9571420,414.220001,0.679881,-0.592633,1.272514,0.0,NaN,0.0,NaN
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,14176944,414.140015,0.621316,-0.520959,1.142274,0.0,0.0,0.0,0.0
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,11437142,413.660004,0.463053,-0.543377,1.006430,0.0,0.0,0.0,0.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,17431474,411.119995,0.146716,-0.687771,0.834487,0.0,0.0,0.0,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,10103321,410.619995,-0.057330,-0.713454,0.656124,0.0,0.0,0.0,0.0


In [43]:
macd_df = macd_df.dropna()

In [44]:
#create a Portfolio Holdings column by multiplying the Close price by the Position

macd_df["Portfolio Holdings"] = macd_df["Close"] * macd_df["Position"]

#review the dataframe
macd_df.head()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings
Date,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,14176944,414.140015,0.621316,-0.520959,1.142274,0.0,0.0,0.0,0.0,0.0
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,11437142,413.660004,0.463053,-0.543377,1.006430,0.0,0.0,0.0,0.0,0.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,17431474,411.119995,0.146716,-0.687771,0.834487,0.0,0.0,0.0,0.0,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,10103321,410.619995,-0.057330,-0.713454,0.656124,0.0,0.0,0.0,0.0,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,5748608,411.250000,-0.284746,-0.752696,0.467950,0.0,0.0,0.0,0.0,0.0


In [45]:
#To calculate Portfolio Cash, subtrace the cumulative sum of the trade cost/proceeds from the initial_capital
#The trade cost proceeds are calculated by multiplying the Close price by the Entry/Exit Position

macd_df["Portfolio Cash"] = initial_capital - (macd_df["Close"] * macd_df["Entry/Exit Position"]).cumsum()

#review the dataframe
macd_df.tail(10)

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash
Date,,,,,,,,,,,,,,
2022-04-11 14:30:00-04:00,440.959991,441.529999,443.000000,10120550,440.859985,-2.135677,-0.548307,-1.587370,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-11 15:30:00-04:00,439.959991,440.950012,441.019989,18938227,439.390015,-2.305212,-0.574274,-1.730938,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 09:30:00-04:00,443.589996,443.079987,445.750000,18679011,442.369995,-2.122195,-0.313006,-1.809190,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 10:30:00-04:00,444.059998,443.600006,444.230011,9075047,442.219910,-1.917129,-0.086351,-1.830778,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 11:30:00-04:00,441.790009,444.070007,444.500000,6698770,441.630005,-1.915698,-0.067937,-1.847762,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,7124676,441.170013,-1.904712,-0.045560,-1.859152,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,10061518,438.420013,-2.125682,-0.213224,-1.912458,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,14347270,436.650085,-2.388655,-0.380958,-2.007697,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,12496573,436.679993,-2.470944,-0.370598,-2.100347,0.0,0.0,0.0,0.0,0.0,102223.382568


In [46]:
#calculate the Portfolo Total by adding Portfolio Cash and Portfolio Holdings
macd_df["Portfolio Total"] = macd_df["Portfolio Cash"] + macd_df["Portfolio Holdings"]

#review the dataframe
macd_df.tail()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
Date,,,,,,,,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,7124676,441.170013,-1.904712,-0.045560,-1.859152,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,10061518,438.420013,-2.125682,-0.213224,-1.912458,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,14347270,436.650085,-2.388655,-0.380958,-2.007697,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,12496573,436.679993,-2.470944,-0.370598,-2.100347,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,0,438.290009,-2.505660,-0.324251,-2.181409,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568


In [47]:
#calculate the Portfolio Period Returns based on the Portfolio Total
macd_df["Portfolio Period Returns"] = macd_df["Portfolio Total"].pct_change()

#review the dataframe
macd_df.tail()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns
Date,,,,,,,,,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,7124676,441.170013,-1.904712,-0.045560,-1.859152,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,10061518,438.420013,-2.125682,-0.213224,-1.912458,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,14347270,436.650085,-2.388655,-0.380958,-2.007697,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,12496573,436.679993,-2.470944,-0.370598,-2.100347,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,0,438.290009,-2.505660,-0.324251,-2.181409,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0


In [48]:
#Calculate the Portfolio Cumulative Returns based on the Portfolio Daily Returns
macd_df["Portfolio Cumulative Returns"] = (1 + macd_df["Portfolio Period Returns"]).cumprod() - 1

#review the dataframe
macd_df.head()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,14176944,414.140015,0.621316,-0.520959,1.142274,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,NaN,NaN
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,11437142,413.660004,0.463053,-0.543377,1.006430,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,17431474,411.119995,0.146716,-0.687771,0.834487,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,10103321,410.619995,-0.057330,-0.713454,0.656124,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,5748608,411.250000,-0.284746,-0.752696,0.467950,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0


## Visualize the MACD Algorithm

In [49]:
#visualize entry positions relative to close price

entry = macd_df[macd_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color = 'purple',
    marker = '^',
    legend = False,
    ylabel = "Price in $",
    width = 1400,
    height = 600)

#visualize the exit positions relative to close price
exit = macd_df[macd_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color = 'orange',
    marker = 'v',
    legend = False,
    ylabel = "Price in $",
    width = 1400,
    height = 600)

#visualize the close price for investment
security_close = macd_df[['Close']].hvplot(
    line_color = "lightblue",
    ylabel = "Price in $",
    width = 1400,
    height = 600)

entry_exit_plot = security_close * entry * exit
entry_exit_plot.opts(title = "MACD Trading Algorithm Entry/Exits")

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [50]:
#visualize entry positions relative to close price

entry = macd_df[macd_df["Entry/Exit"] == 1.0]["Portfolio Total"].hvplot.scatter(
    color = 'purple',
    marker = '^',
    legend = False,
    ylabel = "Total Portfolio Value",
    width = 1400,
    height = 600)

#visualize the exit positions relative to close price
exit = macd_df[macd_df["Entry/Exit"] == -1.0]["Portfolio Total"].hvplot.scatter(
    color = 'orange',
    marker = 'v',
    legend = False,
    ylabel = "Total Portfolio Value",
    width = 1400,
    height = 600)

#visualize the Portfolio Total for investment
total_portfolio_value = macd_df[['Portfolio Total']].hvplot(
    line_color = "lightblue",
    ylabel = "Total Portfolio Value",
    width = 1400,
    height = 600)

portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title = "MACD Trading algorithm - Total Portfolio Value",
    yformatter="%.0f"
)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

## Evaluate the portfolio metrics of the MACD Algorithm

In [51]:
#create a new DataFrame to evaluate the performance of the algorithm

# create a list for the column name

columns = ["Backtest"]

metrics = [

    "Annualized Return",
    "Cumulative Returns",
    "Annual Volatility",
    "Sharpe Ratio"
    ]
portfolio_evaluation_df = pd.DataFrame(index = metrics, columns=columns)

portfolio_evaluation_df.head()

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN


In [52]:
# calculate the annualized return

portfolio_evaluation_df.loc["Annualized Return"] = (

    macd_df["Portfolio Period Returns"].mean() * 252 * 7

)

portfolio_evaluation_df

,Backtest
Annualized Return,0.02322
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN


In [53]:
# calculate the cumulative return

portfolio_evaluation_df.loc["Cumulative Returns"] = (

    macd_df["Portfolio Cumulative Returns"][-1]
)

portfolio_evaluation_df

,Backtest
Annualized Return,0.02322
Cumulative Returns,0.022234
Annual Volatility,NaN
Sharpe Ratio,NaN


In [54]:
# Calculate the annual volatility

portfolio_evaluation_df.loc["Annual Volatility"] = (

    macd_df["Portfolio Period Returns"].std() * np.sqrt(252*7)

)

portfolio_evaluation_df

,Backtest
Annualized Return,0.02322
Cumulative Returns,0.022234
Annual Volatility,0.043342
Sharpe Ratio,NaN


In [55]:
# Calculate the Sharpe Ratio

portfolio_evaluation_df.loc["Sharpe Ratio"] = (

    macd_df["Portfolio Period Returns"].mean() * 252*7) / (

    macd_df["Portfolio Period Returns"].std() * np.sqrt(252*7)

)

portfolio_evaluation_df

,Backtest
Annualized Return,0.02322
Cumulative Returns,0.022234
Annual Volatility,0.043342
Sharpe Ratio,0.535739


## Trade-Level Analytics

In [56]:
macd_df["value"] =  macd_df["Entry/Exit Position"] * macd_df["Close"]

In [63]:
macd_df["Entry/Exit"].value_counts()

 0.0    1606
 1.0      68
-1.0      68
Name: Entry/Exit, dtype: int64

In [57]:
# create a new dataframe to evaluate trade-level Risk/Reward Metrics

macd_trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock',
        'Entry Date',
        'Exit Date',
        'Shares',
        'Entry Share Price',
        'Exit Share Price',
        'Entry Portfolio Value',
        'Exit Portfolio Value',
        'Profit/Loss']
)


In [58]:
#initialize the iterative values
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0.0
entry_share_price = 0.0
exit_share_price = 0.0


In [59]:
# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in macd_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_value = row['value']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_value = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_value - entry_portfolio_value
        macd_trade_evaluation_df = macd_trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Value': entry_portfolio_value,
                'Exit Portfolio Value': exit_portfolio_value,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
macd_trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Value,Exit Portfolio Value,Profit/Loss
0,SPY,2021-04-21 12:30:00-04:00,2021-04-22 14:30:00-04:00,100.0,414.700989,412.174988,41470.098877,41217.498779,-252.600098
1,SPY,2021-04-23 11:30:00-04:00,2021-04-27 10:30:00-04:00,100.0,416.415009,417.195007,41641.500854,41719.500732,77.999878
2,SPY,2021-04-29 13:30:00-04:00,2021-04-30 10:30:00-04:00,100.0,419.290009,416.929993,41929.000854,41692.999268,-236.001587
3,SPY,2021-05-03 13:30:00-04:00,2021-05-03 15:30:00-04:00,100.0,418.934998,418.179993,41893.499756,41817.999268,-75.500488
4,SPY,2021-05-05 11:30:00-04:00,2021-05-10 13:30:00-04:00,100.0,417.183105,419.769989,41718.310547,41976.998901,258.688354
...,...,...,...,...,...,...,...,...,...
63,SPY,2022-03-25 13:30:00-04:00,2022-03-25 14:30:00-04:00,100.0,451.720001,450.070007,45172.000122,45007.000732,-164.999390
64,SPY,2022-03-25 15:30:00-04:00,2022-03-28 11:30:00-04:00,100.0,452.660004,450.619995,45266.000366,45061.999512,-204.000854
65,SPY,2022-03-28 14:30:00-04:00,2022-03-30 12:30:00-04:00,100.0,454.489990,459.000000,45448.999023,45900.000000,451.000977
66,SPY,2022-04-04 11:30:00-04:00,2022-04-05 12:30:00-04:00,100.0,455.565399,454.184998,45556.539917,45418.499756,-138.040161


In [60]:
#determine a winning vs. losing trade

macd_win = 0
macd_loss = 0
macd_max_win = macd_trade_evaluation_df["Profit/Loss"].max()
macd_max_loss = macd_trade_evaluation_df["Profit/Loss"].min()

for pnl in macd_trade_evaluation_df["Profit/Loss"]:
    if pnl > 0:
        macd_win +=1
    else:
        macd_loss +=1

macd_winrate = macd_win / (macd_win + macd_loss)

print(f"There were {macd_win} winning trades and {macd_loss} losing trades giving us a winrate of {macd_winrate})")
print(f"The largest gain made was ${macd_max_win} and the largest loss was ${macd_max_loss}")

There were 26 winning trades and 42 losing trades giving us a winrate of 0.38235294117647056)
The largest gain made was $1857.000732421875 and the largest loss was $-1120.9991455078125


In [61]:
#let's see how much money our system made in total:

total_profit_loss = macd_trade_evaluation_df["Profit/Loss"].sum()

print(f"Using our algorithm made ${total_profit_loss}!")

Using our algorithm made $2223.382568359455!
